In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Masking
from tensorflow.keras.layers import LSTM, GlobalMaxPooling1D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences


data = pd.read_csv('SuperData.csv')

In [2]:
train_sessions, test_sessions = train_test_split(data['label'].unique(), test_size=0.2, random_state=42)
train_data = data[data['label'].isin(train_sessions)]
test_data = data[data['label'].isin(test_sessions)]
# Normalización
scaler = StandardScaler()

# Copia explícita para evitar la advertencia
train_data_copy = train_data.copy()
test_data_copy = test_data.copy()

# Aplicar la transformación a las copias
train_data_copy[['horizontal', 'vertical']] = scaler.fit_transform(train_data_copy[['horizontal', 'vertical']])
test_data_copy[['horizontal', 'vertical']] = scaler.transform(test_data_copy[['horizontal', 'vertical']])

In [3]:
X_train = []
y_train = []
for label, group in train_data.groupby('label'):
    seq = group[['horizontal', 'vertical']].values
    X_train.append(seq)

for label, group in train_data.groupby('label'):
    if 'R' in group['movement_type'].values:
        y_train.append(2)
    elif 'L' in group['movement_type'].values:
        y_train.append(1)
    else:
        y_train.append(0)

X_test = []
y_test = []
for label, group in test_data.groupby('label'):
    seq = group[['horizontal', 'vertical']].values
    X_test.append(seq)

for label, group in test_data.groupby('label'):
    if 'R' in group['movement_type'].values:
        y_test.append(2)
    elif 'L' in group['movement_type'].values:
        y_test.append(1)
    else:
        y_test.append(0)

# Convert X_train, y_train, X_test, and y_test to numpy arrays
X_train = np.array(X_train, dtype=object)
# Convert labels to one-hot vectors
y_train = to_categorical(y_train, num_classes=3)

X_test = np.array(X_test, dtype=object)
y_test = to_categorical(y_test, num_classes=3)

In [8]:
# Define your model
model = Sequential()
model.add(LSTM(50, return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dense(3, activation='softmax'))  # Change number of units and activation function

# Compile your model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Pad the sequences to a fixed length
X_train_padded = pad_sequences(X_train, dtype='float32', padding='post')
X_test_padded = pad_sequences(X_test, dtype='float32', padding='post')

# Now, you can feed the padded sequences into your model
model.fit(X_train_padded, y_train, epochs=20, batch_size=32)

# Evaluate your model
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

Epoch 1/20
4/4 [==============================] - 20s 3s/step - loss: 1.1013 - accuracy: 0.3333
Epoch 2/20
4/4 [==============================] - 11s 3s/step - loss: 1.0822 - accuracy: 0.5641
Epoch 3/20
4/4 [==============================] - 10s 3s/step - loss: 1.0743 - accuracy: 0.5983
Epoch 4/20
4/4 [==============================] - 10s 2s/step - loss: 1.0623 - accuracy: 0.6154
Epoch 5/20
4/4 [==============================] - 12s 3s/step - loss: 1.0524 - accuracy: 0.6154
Epoch 6/20
4/4 [==============================] - 12s 3s/step - loss: 1.0398 - accuracy: 0.6239
Epoch 7/20
4/4 [==============================] - 11s 3s/step - loss: 1.0267 - accuracy: 0.6068
Epoch 8/20
4/4 [==============================] - 12s 3s/step - loss: 1.0079 - accuracy: 0.6068
Epoch 9/20
4/4 [==============================] - 12s 3s/step - loss: 0.9855 - accuracy: 0.6068
Epoch 10/20
3/4 [=====================>........] - ETA: 3s - loss: 0.9787 - accuracy: 0.6146

KeyboardInterrupt: 

In [27]:
# Save the model
model.save('my_modelV2.h5')

c:\Users\Asus\Documents\aUniversidad\SEMESTRE 23-2\Bio_I\Proyecto\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
# Load the new dataset
from keras.models import load_model
probe_data = pd.read_csv('prueba3.csv')
loadmodel = load_model('my_modelV2.h5')

# Preprocess the probe data in the same way as your training data
X_probe = []
for label, group in probe_data.groupby('label'):
    seq = group[['horizontal', 'vertical']].values
    seq = scaler.transform(seq)  # Normalize the data
    X_probe.append(seq)
X_probe = pad_sequences(X_probe, dtype='float32', padding='post')

# Use the model to predict the class labels for the probe data
y_probe_pred = model.predict(X_probe)

# The output, y_probe_pred, is a 2D array with the probabilities for each class.
# To get the predicted class labels, you can use argmax to find the class with the highest probability.
y_probe_pred_labels = np.argmax(y_probe_pred, axis=1)
print(y_probe_pred_labels)

c:\Users\Asus\Documents\aUniversidad\SEMESTRE 23-2\Bio_I\Proyecto\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 339ms/step
[0]


In [4]:
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from keras import regularizers

# Define your model
model = Sequential()
model.add(LSTM(100, return_sequences=True, kernel_regularizer=regularizers.l2(0.01)))  # L2 regularization
model.add(Dropout(0.5))  # Dropout
model.add(GlobalMaxPooling1D())
model.add(Dense(3, activation='softmax'))

# Compile your model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Pad the sequences to a fixed length
X_train_padded = pad_sequences(X_train, dtype='float32', padding='post')
X_test_padded = pad_sequences(X_test, dtype='float32', padding='post')

# Now, you can feed the padded sequences into your model
early_stop = EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train_padded, y_train, epochs=20, batch_size=64, callbacks=[early_stop])

# Evaluate your model
test_loss, test_accuracy = model.evaluate(X_test_padded, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')



Epoch 1/20


4/4 [==============================] - 447s 99s/step - loss: 1.0640 - accuracy: 0.4439
Epoch 2/20
4/4 [==============================] - 448s 101s/step - loss: 0.9669 - accuracy: 0.6020
Epoch 3/20
4/4 [==============================] - 446s 100s/step - loss: 1.0037 - accuracy: 0.6020
Epoch 4/20
4/4 [==============================] - 451s 100s/step - loss: 0.9908 - accuracy: 0.6020
Epoch 5/20
4/4 [==============================] - 2736s 104s/step - loss: 0.9406 - accuracy: 0.6020
Epoch 6/20
4/4 [==============================] - 446s 99s/step - loss: 0.9095 - accuracy: 0.6020
Epoch 7/20
4/4 [==============================] - 450s 100s/step - loss: 0.8808 - accuracy: 0.6020
Epoch 8/20
4/4 [==============================] - 452s 101s/step - loss: 0.8540 - accuracy: 0.6020
Epoch 9/20
4/4 [==============================] - 455s 101s/step - loss: 0.8461 - accuracy: 0.6020
Epoch 10/20
4/4 [==============================] - 2894s 915s/step - loss: 0.8240 - accuracy: 0.6020
Epoch

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv('SuperData.csv')

# Get the unique sessions
sessions = data['label'].unique()

# Plot each session
for session in sessions:
    session_data = data[data['label'] == session]
    
    plt.figure(figsize=(10, 6))
    plt.plot(session_data['horizontal'], label='Horizontal')
    plt.plot(session_data['vertical'], label='Vertical')
    plt.title(f'Session {session}')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.show()

In [17]:
# Load the left and right datasets
left_data = pd.read_csv('LEFT_.csv')
right_data = pd.read_csv('RIGHT_.csv')

# Get the maximum label in the left dataset
max_left_label = right_data['label'].max()

# Increment the labels in the right dataset so they're unique
left_data['label'] += max_left_label

# Append the right dataset to the left dataset
data = pd.concat([right_data, left_data], ignore_index=True)

# Save the combined dataset
data.to_csv('SuperData.csv', index=False)

In [10]:
# Load the extra dataset
extra_data = pd.read_csv('UP_.csv')
data = pd.read_csv('SuperData.csv')

# Get the maximum label in the current data
max_label = data['label'].max()

# Increment the labels in the extra dataset so they're unique
extra_data['label'] += max_label

# Append the extra dataset to the current data
data = pd.concat([data, extra_data], ignore_index=True)

# Save the combined dataset
data.to_csv('SuperData.csv', index=False)